In [ ]:
import glob
import os;
import six.moves.cPickle as pickle
import gzip
import numpy as np
import PIL.Image as Image
import keras
import theano
import matplotlib.pyplot as plt

theano.config.floatX = 'float32'
DATA_LOC=r"C:\Users\NAVPREET\AnacondaProjects\inpainting\inpainting"
FTRAIN   = 'train2014'
FVALID   = 'val2014'
CAPTION  = 'dict_key_imgID_value_caps_train_and_valid.pkl'


def load(batch_idx = None, batch_size = None, valid = False):
    '''Load data from FVALID if *valid* is True, otherwise load from FTRAIN.
       Discard black and white examples by default.
    '''
    # Path to dataset
    fname = FVALID if valid else FTRAIN
    data_path = os.path.join(DATA_LOC, fname)
    print data_path + "/*.jpg"
    imgs = glob.glob(data_path + "/*.jpg")
    if batch_idx != None:
        batch_imgs = imgs[batch_idx*batch_size:(batch_idx+1)*batch_size]
    else:
        batch_imgs = imgs

    # Initialize variables
    X_int, y_int= [], []

    for i, img_path in enumerate(batch_imgs):
    # loading images from the given path
        img = Image.open(img_path)
        img_array = np.array(img)

        # Get input/target from the images
        center = (int(np.floor(img_array.shape[0] / 2.)), int(np.floor(img_array.shape[1] / 2.)))
        if len(img_array.shape) == 3:
            input = np.copy(img_array)
            input[center[0]-16:center[0]+16, center[1]-16:center[1]+16, :] = 0
            target = img_array[center[0]-16:center[0]+16, center[1] - 16:center[1]+16, :]
        
        X_int.append(input)
        y_int.append(target)

    X = np.asarray(X_int, dtype=np.float32) / 255.
    y = np.asarray(y_int, dtype=np.float32) / 255.
    X = np.transpose(X, (0,3,1,2))
    y = np.transpose(y, (0,3,1,2))

    return X, y



def rebuilt_image(X, ypd, display = True): 
    
    # Rearrange shapes, data type
    X = np.asarray(255*X.transpose(0,2,3,1), np.uint8)
    ypd = np.asarray(255*ypd.transpose(0,2,3,1), np.uint8)

    collage = []

    for i in range(len(X)):
        img_array = X[i,:]
        center = (int(np.floor(img_array.shape[0] / 2.)), int(np.floor(img_array.shape[1] / 2.)))
        # Draw predicted image
        imgpd = np.copy(img_array)
        imgpd[center[0]-16:center[0]+16, center[1]-16:center[1]+16, :] = ypd[i,:]
        if i == 0: collage = imgpd
        else:      collage = np.hstack((collage, imgpd))

    # Display images
    if display:
        Image.fromarray(collage).show()


if __name__ == '__main__':
    X_train, y_train=load(1, 400)
    X_val, y_val=load(2, 10, valid=True)
    #img_train = draw(X[0:n,], y[0:n,], ysample[0:n], display=False, show_gt=True, return_array=True)



Using Theano backend.


C:\Users\NAVPREET\AnacondaProjects\inpainting\inpainting\train2014/*.jpg
Examples loaded: 1000
C:\Users\NAVPREET\AnacondaProjects\inpainting\inpainting\val2014/*.jpg
Examples loaded: 10


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Dropout, MaxPooling2D, Deconvolution2D, UpSampling2D, Reshape, Input
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from keras.models import load_model, Model
import scipy as sp
from numpy import squeeze
# defining hyperparameters of model
nb_epoch=10 
        
        
model=Sequential()
nb_epoch=1
batch_size=128
nb_filters=10
nb_pool=2
nb_conv=2
sgd=SGD(lr=0.2, decay=1e-6, momentum=0.9, nesterov=True)


# adding layers to model
model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='same', input_shape=(3, 64, 64), activation='relu', strides=(1,1), data_format="channels_first"))
model.add(Activation('relu'))
model.add(Convolution2D(filters=16, kernel_size=(3,3), padding='same', activation='relu', strides=(1,1), data_format="channels_first"))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Convolution2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', strides=(1,1), data_format="channels_first"))
model.add(Activation('relu'))
model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='same', strides=(1,1), activation='relu', data_format="channels_first"))
model.add(Activation('relu'))
model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='same', strides=(1,1), activation='relu', data_format="channels_first"))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(6912))
model.add(Activation('relu'))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(3072))
model.add(Activation('relu'))


model.add(Reshape([3,32, 32]))
#model.add(UpSampling2D([2, 2], data_format="channels_first"))
model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='same', strides=(1,1), activation='relu', data_format="channels_first"))
model.add(Convolution2D(filters=18, kernel_size=(5,5), padding='same', strides=(1,1), activation='relu', data_format="channels_first"))
model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='same', strides=(1,1), activation='relu', data_format="channels_first"))
model.add(Convolution2D(filters=16, kernel_size=(3,3), padding='same', strides=(1,1), activation='relu', data_format="channels_first"))
#model.add(UpSampling2D([2,2]))
model.add(Convolution2D(filters=3, kernel_size=(3,3), padding='same', strides=(1,1), activation='relu', data_format="channels_first"))
 #do all calculations do see size of all inputs and outputs
model.add(Reshape([3, 32, 32]))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
history= model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=0, validation_data=(X_val, y_val))
score=model.evaluate(X_val, y_val, verbose=0)
print('Test Score:', score[0] )
print('accuracy:', score[1])
plt.show()


    


In [ ]:
im, im_target=load(1,1)
imv, imv_tar=load(1,1, valid=True)
imv.reshape(64, 64, 3)
imv_tar.reshape(32, 32, 3)
pr = model.predict(im)
pr.reshape(32, 32, 3)
rebuilt_image(im, pr, save_filename='abd1.png')
plt.show()